In [ ]:
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import PIL.Image
import cv2
from PIL import Image

In [ ]:
tf.__version__

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top = False, weights = 'imagenet')

In [ ]:
base_model.summary()

In [ ]:
img_1 = Image.open('/content/drive/MyDrive/Datasets/Creative AI Dataset/mars.jpg')

img_2 = Image.open('/content/drive/MyDrive/Datasets/Creative AI Dataset/eiffel.jpg')

image = Image.blend(img_1, img_2, 0.5)

image.save("img_0.jpg")

In [ ]:
Sample_Image = tf.keras.preprocessing.image.load_img('img_0.jpg')

In [ ]:
Sample_Image

In [ ]:
np.shape(Sample_Image)

In [ ]:
type(Sample_Image)

In [ ]:
Sample_Image = tf.keras.preprocessing.image.img_to_array(Sample_Image)
#Sample_Image = np.array(Sample_Image)

In [ ]:
type(Sample_Image)

In [ ]:
print("Min pixel value = {}, Max pixel value = {}".format(Sample_Image.min(), Sample_Image.max()))

In [ ]:
Sample_Image = np.array(Sample_Image)/255.0
Sample_Image.shape

In [ ]:
print("Min pixel value = {}, Max pixel value = {}".format(Sample_Image.min(), Sample_Image.max()))

In [ ]:
Sample_Image = tf.expand_dims(Sample_Image, axis = 0)

In [ ]:
np.shape(Sample_Image)

In [ ]:
plt.imshow(np.squeeze(Sample_Image))

In [ ]:
names = ['mixed3', 'mixed5','mixed7']

layers = [base_model.get_layer(name).output for name in names]

deepdream_model = tf.keras.Model(inputs = base_model.input, outputs = layers)

In [ ]:
deepdream_model.summary()

In [ ]:
activations = deepdream_model(Sample_Image)
activations

In [ ]:
len(activations)

In [ ]:
Sample_Image = tf.squeeze(Sample_Image, axis=0)

In [ ]:
def calc_loss(image,model):
  img_batch = tf.expand_dims(image, axis=0)
  layer_activations = model(img_batch)
  print('ACTIVATION VALUES (LAYER OUTPUT) =\n', layer_activations)

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  print('LOSSES (FROM MULTIPLE ACTIVATION LAYERS) = ', losses)
  print('LOSSES SHAPE (FROM MULTIPLE ACTIVATION LAYERS) = ', np.shape(losses))
  print('SUM OF ALL LOSSES (FROM ALL SELECTED LAYERS)= ', tf.reduce_sum(losses))

  return  tf.reduce_sum(losses) # Calculate sum 

In [ ]:
loss = calc_loss(tf.Variable(Sample_Image), deepdream_model)

In [ ]:
loss

In [ ]:
@tf.function
def deepdream(model, image, step_size):
  with tf.GradientTape() as tape:
    tape.watch(image)
    loss = calc_loss(image, model)

  gradients = tape.gradient(loss, image)
  print('Gradients = \n', gradients)  
  print('Gradients Shape = \n', np.shape(gradients))  

  gradients /= tf.math.reduce_std(gradients)
  image = image + gradients*step_size
  image = tf.clip_by_value(image, -1, 1)

  return loss, image

In [ ]:
def run_deep_dream_simple(model, image, steps = 100, step_size = 0.01):
  image = tf.keras.applications.inception_v3.preprocess_input(image)

  for step in range(steps):
    loss, image = deepdream(model, image, step_size)

    if step % 100 == 0:
      plt.figure(figsize=(12,12))
      plt.imshow(deprocess(image))
      plt.show()
      print("Step {}, loss {}".format(step, loss)) 

  plt.figure(figsize=(12,12))
  plt.imshow(deprocess(image))
  plt.show()

  return deprocess(image)

In [ ]:
def deprocess(image):
  image = 255*(image + 1.0)/2.0
  return tf.cast(image, tf.uint8)

In [ ]:
Sample_Image = np.array(tf.keras.preprocessing.image.load_img('img_0.jpg'))
dream_img = run_deep_dream_simple(model = deepdream_model, image = Sample_Image, steps = 4000, step_size = 0.001)

In [ ]:
image = tf.keras.preprocessing.image.load_img("img_0.jpg")

In [ ]:
dream_name = 'mars_eiffel'

In [ ]:
x_size = 910 
y_size = 605
created_count = 0
max_count = 50

In [ ]:
def load_image(filename):
    image = PIL.Image.open(filename)
    return np.float32(image)

In [ ]:
for i in range(0, 50):     
    if os.path.isfile('/content/drive/MyDrive/Datasets/Creative AI Dataset/{}/img_{}.jpg'.format(dream_name, i+1)):
        print("{} present already, continue fetching the frames...".format(i+1))
        
    else:
        # Call the load image funtion
        img_result = load_image(r'/content/drive/MyDrive/Datasets/Creative AI Dataset/{}/img_{}.jpg'.format(dream_name, i))

    
        # Zoom the image 
        x_zoom = 2 # this indicates how quick the zoom is 
        y_zoom = 1
        
        # Chop off the edges of the image and resize the image back to the original shape. This gives the visual changes of a zoom
        img_result = img_result[0+x_zoom : y_size-y_zoom, 0+y_zoom : x_size-x_zoom]
        img_result = cv2.resize(img_result, (x_size, y_size))
        
        # Adjust the RGB value of the image
        img_result[:, :, 0] += 2  # red
        img_result[:, :, 1] += 2  # green
        img_result[:, :, 2] += 2  # blue
        
        # Deep dream model  
        img_result = run_deep_dream_simple(model = deepdream_model, image = img_result, steps = 500, step_size = 0.001)
        
        # Clip the image, convert the datatype of the array, and then convert to an actual image. 
        img_result = np.clip(img_result, 0.0, 255.0)
        img_result = img_result.astype(np.uint8)
        result = PIL.Image.fromarray(img_result, mode='RGB')
        
        # Save all the frames in the dream location
        result.save(r'/content/drive/MyDrive/Datasets/Creative AI Dataset/{}/img_{}.jpg'.format(dream_name, i+1))
        
        created_count += 1
        if created_count > max_count:
            break

In [ ]:
dream_path = '/content/drive/MyDrive/Datasets/Creative AI Dataset/mars_eiffel'

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter('deepdreamvideo.avi', fourcc , 5.0, (910, 605))

In [ ]:
for i in range(9999999999999):
    if os.path.isfile('/content/drive/MyDrive/Datasets/Creative AI Dataset/mars_eiffel/img_{}.jpg'.format(i+1)):
        pass
    else:
        dream_length = i
        break

In [ ]:
for i in range(dream_length):
    
    # Build the frames of cv2.VideoWriter
    img_path = os.path.join(dream_path,'img_{}.jpg'.format(i)) # join the dream path
    
    print(img_path) # print the image path 
    
    frame = cv2.imread(img_path)
    out.write(frame)

out.release()